# Semantic Search
Idea: Use text embeddings to find the example that is most similar to the translation.

Model: https://huggingface.co/intfloat/multilingual-e5-small

In [31]:
def format_translation(source, target):
    # Format according to XLIFF
    return f"query: <source>{source}</source><target>{target}</target>"


def format_example(ex):
    # Use "query: " prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.
    return f"query: {ex}"

In [5]:
from find_examples import CorpusExamples

corpus_folder = "/media/ducha/SSDSHARED/VN/subs_dump/viet_subs_processed2"
corpus = CorpusExamples(corpus_folder)
csv_path = "/home/ducha/Dropbox/Tiếng Việt/Ngữ Vựng/1000.csv"

with open(csv_path, "r") as f:
    csv_1k = [line.strip().split(";") for line in f]

Preparing corpus from /media/ducha/SSDSHARED/VN/subs_dump/viet_subs_processed2
Corpus prepared with 13646 files
Total Examples 7736833


In [14]:
vi, en, _ = csv_1k[4]

In [15]:
query = format_translation(vi, en)
print(query)

query: <source>cột</source><target>column</target>


In [ ]:
found_examples = corpus.find_examples(vi)
len(found_examples)

## Model Inference

In [34]:
examples_formatted = [format_example(ex["text"]) for ex in found_examples]

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-small").to("cuda")

In [45]:
model_input = [query] + examples_formatted
print(len(model_input))
all_encoded = model.encode(
    model_input,
    normalize_embeddings=True,
    device="cuda",
    convert_to_tensor=True,
    convert_to_numpy=False,
)

1100


In [46]:
query_enc, examples_enc = all_encoded[0], all_encoded[1:]

In [65]:
import torch

similarities = torch.cosine_similarity(query_enc[None], examples_enc, dim=1)
similarities.shape

torch.Size([1099])

In [71]:
similarities_sorted = (
    torch.argsort(similarities, descending=True).cpu().detach().numpy()
)

In [76]:
import numpy as np

found_ex_np = np.array([ex["text"] for ex in found_examples])

In [77]:
found_ex_np[similarities_sorted]

array(['Đã cột và vào vị trí.', 'Cột ở phía này trước đi.',
       'Cắt ở đây, cột ở chỗ kia.', ...,
       'Ai không theo Kitô giáo được lọc ra, bị treo trên cột và bị thiêu chết?',
       'Chúng ta sẽ cột tàu ở đó Jack có thể thấy chúng ta An toàn không?',
       'Phelps được tìm thấy từ đỉnh cột cờ, có vẻ như cổ của anh ta Có nhân chứng không?'],
      dtype='<U92')